In [1]:
#Jesus Christ died to redeem my body and soul from the wrath to come and seat me beside Him in the throneroom of the Most High.
import requests, json, base64
import os
import re
from base64 import b64encode
from shutil import copyfile

####
#### This file is a template for document transcription using Google Cloud OCR.
#### At its simplest, insert a security code from a Google Cloud Vision API account in the KEY position.
#### Then 
####
 
### Cell #1

#For an image filepath, get the OCR .json response from Google Cloud Vision API
def extractTextJpg(documentFilename):
    #Extract an encoded version of the file. So far it works for .jpg.
    encoding = 'utf-8'
    with open(documentFilename, 'rb') as open_file:
        byte_content = open_file.read()
    base64_bytes = b64encode(byte_content)
    base64_string = base64_bytes.decode(encoding)
    #Make the request json, incorporating the encoded document. 
    myJson =   {
      "requests": [
        {

          "features": [
            {
              "type": "DOCUMENT_TEXT_DETECTION"
            }
          ],
          "image": {
            "content": base64_string     
          }
        }
      ]
    }
    #Place the key as a parameter and then POST a job.
    parameters = { "key": "INSERT_SECURITY_KEY_HERE"}  
    #The security key should be ~40 characters long, numbers and letters
    response = requests.post("https://vision.googleapis.com/v1/images:annotate",params = parameters, json=myJson)
    return response           



#This function 1) sends a picture to Google Cloud OCR
#              2) It saves the complete OCR response as a .json file alongside the initial file
#              3) It saves the extracted text as a .txt file alongside the initial file
#                     This extracted text has a) newlines replaced by space and 2) "- " replaced by nothing.
#                     This is because newlines and also hyphens over line breaks are kept literally by the OCR.
def transcribe_a_picture_and_save_TXT(documentFilePath):
    response = extractTextJpg(documentFilePath)
    assert response.status_code == 200, "Server status code != 200. Some problem."
    data = response.json()
    #print(data)
    with open(documentFilePath +'.json', 'w') as outfile:
        json.dump(data, outfile)
    with open(documentFilePath +'.txt', 'w') as outfile:
        myTranscript = data['responses'][0]['fullTextAnnotation']['text']
        myTranscript = myTranscript.replace('\n', ' ')
        myTranscript = myTranscript.replace('- ', '')
        outfile.write(myTranscript)
    print(documentFilePath+" is now written to .json and .txt.")

    
#For a list of folders, collect all the filepaths inside each of the folders that end in a file ending, 
#     for example, ending could equal".jpg" or ".png"
def filesOfEnding(folders,ending):
    rr = []
    justNames = []
    for folder in folders:
        for f in os.listdir(folder):
            if re.search(ending, f):
                rr += [folder + "/" + f] #[each for each in os.listdir(folder) if each.endswith(ending)]
                justNames += [f]
    return rr,justNames

# This function gets a list of the folders within a given folder. 
# It is useful for passing a list of folders into the filesOfEnding function.
def getFolders(superFolder):
    folders = []
    for f in os.listdir(superFolder):
        folders.append(superFolder + "/" + f)
    return folders
    
# Take a list of folder paths.
# Return a list of all the image files (of ending ".png") that have not previously been transcribed.
# This only works for png and jpg so far. Switch in the file ending if working with another type. 
def get_FilePaths_from_a_Folder_for_Images_Not_Already_Transcribed(folders):
    filesJpg1,trash = filesOfEnding(folders,'.png')  
    filesTxt1,trash = filesOfEnding(folders,'.png.txt')
    
    addendumJpg1,trash = filesOfEnding(folders,'.jpg')  
    addendumTxt1,trash = filesOfEnding(folders,'.jpg.txt')
    
    filesJpg1.extend(addendumJpg1)
    filesTxt1.extend(addendumTxt1)
    
    filesTxt = []
    for item in filesTxt1:
        filesTxt.append(item[0:-4])
    filesJpg = []
    for item in filesJpg1:
        good = True
        for completed in filesTxt:
            if completed in item:
                good = False
        if good == True:
            filesJpg.append(item)
        #if item not in filesTxt:
         #   filesJpg.append(item)
    print("Examples of files to be transcribed: ")
    for i in range(0,10):
        print(filesJpg[i])
    print()
    return filesJpg


###
### THIS IS LEFTOVER CODE. Don't use it unless you really really mean to.
###
    
foldersClean = ["C:/Users/dforman44/Documents/Churchill 02_/MoreDocs/02_074"]
myUpstream = "C:/Users/dforman44/Documents/Churchill 02_/MoreDocs/myJPGs/"

def transferDysfunct():
    paths,names = filesOfEnding(foldersClean,'.jpg')
    for i in range(len(paths)):
        myPathName = paths[i]
        myNewName = myUpstream+names[i]
        copyfile(myPathName, myNewName)
        os.remove(myPathName)

#This is a dangerous function. Don't call it unless you want to delete a lot of files of these endings.
def merelyJpgs(foldersClean, badEnds = [".tif",".txt",".pdf"]):
    pathsToDelete = []
    for badEnd in badEnds:
        paths, names = filesOfEnding(foldersClean,badEnd)
        pathsToDelete += paths
    for item in pathsToDelete:
        os.remove(item)    

In [18]:
###
### Cell #2
###  Run this cell in order to OCR/transcribe a single image. 
###  It will output a .txt file alonside the original image, as well as a .json file with the full OCR response.
###


# For example:
#filePath = "/Users/davidforman/Box/Churchill Reader project/Forman_Scratch_Materials/tech.png"

filePath = "INSERT_IMAGE_FILEPATH_HERE"
extractTextJpg(filePath)



In [3]:
###
### Cell #3

#superFolder_ = "E:/JaffaJpgsDF/Box06"
#folders = getFolders(superFolder_)

# You could potentially directly use 
#                       folders = ["folderPath1",  "folderPath2", "etc folderpath"]

#Example: 

#folders = ["/Users/davidforman/Box/Churchill Reader project/Forman_Scratch_Materials"]
folders = ["INSERT_IMAGE_FOLDER_PATH_HERE"]
    
    
#This gets the .png and .jpg images from within a list of folders. 
filesJpg = get_FilePaths_from_a_Folder_for_Images_Not_Already_Transcribed(folders)

for documentFilePath in filesJpg:
    transcribe_a_picture_and_save_TXT(documentFilePath)